In [1]:
# This code manually converts the csv files into parquet files after it has been manually downloaded and uploads to redshift
import numpy as np
import pandas as pd
import os

In [ ]:
all_files = os.listdir("/Users/guestt/Desktop/Data Engineering/DE-ZC-DATA/Green Data")    
green_csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
green_csv_files = ["/Users/guestt/Desktop/Data Engineering/DE-ZC-DATA/Green Data/" + f for f in green_csv_files]

In [ ]:
all_files = os.listdir("/Users/guestt/Desktop/Data Engineering/DE-ZC-DATA/Yellow Data")    
yellow_csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
yellow_csv_files = ["/Users/guestt/Desktop/Data Engineering/DE-ZC-DATA/Yellow Data/" + f for f in yellow_csv_files]

In [ ]:
def parse_data(csv_list):
    df_list = []
    for ls in csv_list:
        df = pd.read_csv(ls)
        df_list.append(df)
        
    df = pd.concat(df_list)
    
    if "tpep_pickup_datetime" in df.columns:
        df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
        df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

    if "lpep_pickup_datetime" in df.columns:
        df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])
        df["lpep_dropoff_datetime"] = pd.to_datetime(df["lpep_dropoff_datetime"])
        
    df = df.reset_index(drop=True)
    
    return df

In [ ]:
green_df = parse_data(green_csv_files)

In [ ]:
main_df = pd.read_csv("/Users/guestt/Desktop/Data Engineering/DE-ZC-DATA/Yellow Data/yellow_tripdata_2019-04.csv")

In [ ]:
def parse_data(csv_list):
    
    for ls in csv_list:
        df = pd.read_csv(ls)
        
        if "tpep_pickup_datetime" in df.columns:
            df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
            df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
            
            for x in df.columns:
                if main_df[x].dtypes.name == "int64":
                    df[x]=df[x].astype("Int64")
                else:
                    df[x]=df[x].astype(main_df[x].dtypes.name)

        if "lpep_pickup_datetime" in df.columns:
            df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])
            df["lpep_dropoff_datetime"] = pd.to_datetime(df["lpep_dropoff_datetime"])

        df = df.reset_index(drop=True)
        path = "/Users/guestt/Desktop/Data Engineering/DE-ZC-DATA/Yellow Parquet/" + ls.split("/")[-1].replace(".csv", ".parquet")

        df.to_parquet(path, compression="gzip")

In [ ]:
parse_data(yellow_csv_files)

#### Create Schema to house tables  
create schema trips_data_all  

#### SQL Code to load data into redshift schema from s3  

copy  nytaxi.trips_data_all.yellow_trip_data  
from 's3://zoom-s3-ayo/data/yellow/'  
access_key_id ACCESS_KEY  
secret_access_key SECRET_KEY  
FORMAT AS PARQUET;  


copy  nytaxi.trips_data_all.green_trip_data  
from 's3://zoom-s3-ayo/data/green/'  
access_key_id ACCESS_KEY  
secret_access_key SECRET_KEY  
FORMAT AS PARQUET;  